## Imports

In [ ]:
# import pyspark functions
from pyspark.sql.functions import *
# import URL processing
import urllib

## AWS credentials

In [ ]:
# list tables in filestore to get name of credentials file
dbutils.fs.ls("/FileStore/tables")

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

In [ ]:
# Read the CSV file to spark dataframe, passing in options for the header row and separator
aws_keys_df = spark.read.format("csv")\
.option("header", "true")\
.option("sep", ",")\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
import urllib.parse

# Get the AWS access key and secret key from the spark dataframe for 'databricks-user'
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']

# Encode the secret key for URL safety
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


## Mount S3 bucket

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e7f89a006e1-bucket"

# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e7f89a006e1-bucket"

# Source URL for accessing the bucket, constructed using AWS credentials and bucket name
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

# Mount the S3 bucket as a drive using Databricks' file system utilities (dbutils)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


Out[5]: True

In [ ]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-0e7f89a006e1-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.geo/,1215be80977f.geo/,0,1692041344013
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.pin/,1215be80977f.pin/,0,1692041344013
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.user/,1215be80977f.user/,0,1692041344013


## Reading the contents of S3 bucket into dataframes

In [ ]:
# List of topic suffixes
topics = [".pin", ".geo", ".user"]

def read_topics_into_dataframe(topic):
    # Create path to topic files based on the given topic suffix
    file_path = f"/mnt/user-0e7f89a006e1-bucket/topics/1215be80977f{topic}/partition=0/*.json"
    # Specify file type as JSON
    file_type = "json"
    # Ask Spark to infer the schema while loading data
    infer_schema = "true"
    # Load JSON files from the mounted S3 bucket into a Spark DataFrame
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_path)
    return df

# Iterate through the list of topics and load data into DataFrames
for item in topics:
    # Create strings for naming and displaying dataframes
    make_df_statement = f"df_{item[1:]} = read_topics_into_dataframe('{item}')"
    display_df_statement = f"display(df_{item[1:]})"
    # Execute the statements to create DataFrames and display their content
    exec(make_df_statement)
    exec(display_df_statement)


## Unmounting S3 bucket

In [ ]:
# unmount the bucket from the filestore
dbutils.fs.unmount("/mnt/user-0e7f89a006e1-bucket")

/mnt/user-1215be80977f-bucket has been unmounted.
Out[8]: True